In [ ]:
---
title: Filtracja modelu BPMN
authors: Paweł Bucior, Julia Witek
show-code: False
params:
    file_name:
        input: file
        label: Upload file to generate BPMN model
    activity:
        input: text
        label: Please provide activity label
    start_time:
        input: text
        label: Please provide time label
    case_id:
        input: text
        label: Please provide caseID label
    event_filter:
        input: slider
        value: 700
        min: 0
        max: 1200
        label: Please select event filter value
    flow_filter:
        input: slider
        value: 700
        min: 0
        max: 1200
        label: Please select flow filter value
---

SyntaxError: ignored

In [1]:
!pip install opyenxes
!apt install libgraphviz-dev
# !apt install graphviz graphviz-dev
!pip install pygraphviz
from IPython.display import clear_output

clear_output(wait=False)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgraphviz-dev is already the newest version (2.40.1-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
file_name='/content/repairExample.csv'
case_id=''
start_time=''
activity=''
event_filter=700
flow_filter=700

In [3]:
import pygraphviz as pgv
from itertools import product
from collections import defaultdict
from typing import Dict, Set
from IPython.display import Image, display


In [4]:
from opyenxes.data_in.XUniversalParser import XUniversalParser
import pandas as pd
import xml
from dateutil.parser import parse
from xml.dom import minidom

def is_date(string, fuzzy=False):
    try:
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False

def define_csv_columns(file_name):
  df = pd.read_csv(file_name)
  global case_id, start_time, activity
  if (case_id == '' or start_time == '' or activity == ''):

    if(len(df.columns)<3):
      print("Provided file cannot be used as an event log")

    elif(len(df.columns)==3):
      for col in df.columns:
        
        if (df[col].dtypes == 'int64'):
          case_id = col
        elif (is_date(df.loc[0, col]) == True):
          start_time = col
        elif (df[col].dtypes != 'int64' and is_date(df.loc[0, col]) == False):
          activity = col
        else:
           case_id = ''
           activity = ''
           start_time = ''

    else:
      case_id = ''
      # activity = ''

      for col in df.columns:
         if (df[col].dtypes == 'int64'):
           case_id = col
         elif (start_time != '' and is_date(df.loc[0, col]) == True):
           start_time = col
         else:
           break


  # if (case_id == ''):
  #   case_id = input("Enter the name of id column: ")
  # if (start_time == ''):
  #   start_time = input("Enter the name of timestamp column: ")
  # if (activity == ''):
  #   activity = input("Enter the name of activity column: ")
       
  # print(start_time)
  # print(case_id)
  # print(activity)
  return case_id, activity, start_time


def load_csv(file_name):
  df = pd.read_csv(file_name)
  global case_id, activity, start_time
  case_id, activity, start_time = define_csv_columns(file_name)
  dfs = df[[case_id, activity, start_time]]
  dfs = (df
    .sort_values(by=[case_id, start_time])
    .groupby([case_id])
    .agg({activity: ';'.join})    
  )

  dfs['trace'] = [trace.split(';') for trace in dfs['Activity']]

  events_csv = []
  for i, row in dfs[['trace']].iterrows():
    events_csv.append(row['trace'])

  return events_csv, df, case_id, activity, start_time


def load_xes(file_name):
  mydoc = minidom.parse(file_name)
  traces = mydoc.getElementsByTagName('trace')
    
  df = pd.DataFrame()
    
  case_id = 0
  for trace in traces:
    case_id += 1
    for event in trace.getElementsByTagName('event'):
      events = {}
      for e in event.childNodes:
        if e.nodeType == xml.dom.Node.TEXT_NODE:
          continue
        events[e.attributes['key'].value] = e.attributes['value'].value
                   
      common_entry = {
                "Activity": [events['Activity']],
                "Case ID": [case_id],
                "Start Timestamp": [pd.to_datetime(events['time:timestamp'])]
      }
      df_entry = pd.DataFrame.from_dict(common_entry)
      df = pd.concat([df, df_entry], ignore_index=True)

  with open(file_name) as log_file:
  # parse the log
    log = XUniversalParser().parse(log_file)[0]
    event = log[0][0]
    event.get_attributes()
    events_xes = []
    for trace in log:
        events_w = []
        for event in trace[0::]:
            name = event.get_attributes()['Activity'].get_value()
            events_w.append(name)
        events_xes.append(events_w)
  return events_xes, df


def load_eventlog(file_name):
  f = open(file_name, "r")
  if f.name.endswith('.csv'):
    print('csv') 
    return load_csv(file_name)
  elif f.name.endswith('.xes'):
    print('xes')
    return load_xes(file_name)
  else:
    print('Nie rozpoznano pliku')


In [7]:
def get_start_set_events(events_tab):
  start_set_events = set()
  for line in events_tab:
    start_set_events.add(line[0])

  return start_set_events


def get_end_set_events(events_tab):
  end_set_events = set()
  for line in events_tab:
    end_set_events.add(line[len(line)-1])

  return end_set_events

def get_direct_succession(events):
  direct_new = {}
  unique_elements = []
  for r in events:
   for c in r:
      if c not in unique_elements:
        unique_elements.append(c)
  unique_elements = sorted(unique_elements)
  for ue in unique_elements:
    relations = set()
    for r in events:
      for index, c in enumerate(r):
        if c == ue and int(len(r)) > int(index+1):
          relations.add(r[index+1])
    if len(relations) > 0:
      direct_new[ue] = relations
    else:
      direct_new[ue] = {}

  return direct_new


def get_causality(direct_succession) -> Dict[str, Set[str]]:
    causality = defaultdict(set)
    for ev_cause, events in direct_succession.items():
        for event in events:
            if ev_cause not in direct_succession.get(event, set()):
                causality[ev_cause].add(event)

    return dict(causality)


def get_potential_parallelism(direct_succession) -> Dict[str, Set[str]]:
  potential_parallelism = []
  for ev_cause, events in direct_succession.items():
    tmp = []
    for event in events:
      if ev_cause in direct_succession.get(event, set()):
        if ev_cause not in tmp:
          tmp.append(ev_cause)
        tmp.append(event)

    if len(tmp) > 0:
      potential_parallelism.append(tuple(tmp))
  if len(potential_parallelism) > 0:
    potential_parallelism.append(tuple(sorted(potential_parallelism[0])))

  return potential_parallelism


def get_inv_causality(causality) -> Dict[str, Set[str]]:
    inv_causality = defaultdict(set)
    for key, values in causality.items():
        for value in values:
          inv_causality[value].add(key)

    return {k: v for k, v in inv_causality.items() if len(v) > 1}

In [8]:
class MyGraph(pgv.AGraph):

    def __init__(self, *args):
        super(MyGraph, self).__init__(strict=False, directed=True, *args)
        self.graph_attr['rankdir'] = 'LR'
        self.node_attr['shape'] = 'Mrecord'
        self.graph_attr['splines'] = 'ortho'
        self.graph_attr['nodesep'] = '0.8'
        self.edge_attr.update(penwidth='2')

    def add_event(self, name):
        super(MyGraph, self).add_node(name, shape="circle", label="")

    def add_event2(self, name, **args): #
        super(MyGraph, self).add_node(name, shape="box", **args) #

    def get_node(self,name):
        return super(MyGraph, self).get_node(name)

    def get_edges(self):
        return super(MyGraph, self).edges()

    def add_edge2(self, s, t, style): 
        super(MyGraph, self).add_edge(s,t,penwidth=style[0],label=style[1]) 

    def add_end_event(self, name):
        super(MyGraph, self).add_node(name, shape="circle", label="",penwidth='3')

    def add_and_gateway(self, *args):
        super(MyGraph, self).add_node(*args, shape="diamond",
                                  width=".7",height=".7",
                                  fixedsize="true",
                                  fontsize="40",label="+")

    def add_xor_gateway(self, *args, **kwargs):
        super(MyGraph, self).add_node(*args, shape="diamond",
                                  width=".7",height=".7",
                                  fixedsize="true",
                                  fontsize="40",label="×")
    def add_xor_inclusive_gateway(self, *args, **kwargs):
        super(MyGraph, self).add_node(*args, shape="diamond",
                                  width=".7",height=".7",
                                  fixedsize="true",
                                  fontsize="40",label="o")

    def add_event_gateway(self, source, targets, *args):
        gateway = 'EVENTs '+str(source)+'->'+str(targets)
        self.add_xor_inclusive_gateway(gateway,*args)
        super(MyGraph, self).add_edge(source, gateway)
        for target in targets:
            super(MyGraph, self).add_edge(gateway, target)

    def add_and_split_gateway(self, source, targets, *args):
        gateway = 'ANDs '+str(source)+'->'+str(targets)
        self.add_and_gateway(gateway,*args)
        super(MyGraph, self).add_edge(source, gateway)
        for target in targets:
            super(MyGraph, self).add_edge(gateway, target)

    def add_and_split_gateway2(self, source, targets, left_style, right_style, *args):
        gateway = 'ANDs '+str(source)+'->'+str(targets)        
        self.add_and_gateway(gateway,*args)
        self.add_edge2(source, gateway, left_style)
        for target, style in zip(targets,right_style):
            self.add_edge2(gateway, target, style)
            

    def add_xor_split_gateway(self, source, targets, *args):
        gateway = 'XORs '+str(source)+'->'+str(targets)
        self.add_xor_gateway(gateway, *args)
        super(MyGraph, self).add_edge(source, gateway)
        for target in targets:
            super(MyGraph, self).add_edge(gateway, target)

    def add_xor_split_gateway2(self, source, targets, left_style, right_style, *args):
        gateway = 'XORs '+str(source)+'->'+str(targets) 
        self.add_xor_gateway(gateway, *args)
        self.add_edge2(source, gateway, left_style)
        for target, style in zip(targets,right_style):
            self.add_edge2(gateway, target, style)

    def add_event_merge_gateway(self, sources, target, *args):
        gateway = 'EVENTm '+str(sources)+'->'+str(target)
        self.add_xor_inclusive_gateway(gateway,*args)
        super(MyGraph, self).add_edge(gateway,target)
        for source in sources:
            super(MyGraph, self).add_edge(source, gateway)

    def add_and_merge_gateway(self, sources, target, *args):
        gateway = 'ANDm '+str(sources)+'->'+str(target)
        self.add_and_gateway(gateway,*args)
        super(MyGraph, self).add_edge(gateway,target)
        for source in sources:
            super(MyGraph, self).add_edge(source, gateway)

    def add_and_merge_gateway2(self, sources, target, left_style, right_style, *args):
        gateway = 'ANDm '+str(sources)+'->'+str(target)
        self.add_and_gateway(gateway,*args)
        self.add_edge2(gateway,target, right_style)
        for source, style in zip(sources,left_style):
            self.add_edge2(source,gateway,style)

    def add_xor_merge_gateway(self, sources, target, *args):
        gateway = 'XORm '+str(sources)+'->'+str(target)
        self.add_xor_gateway(gateway, *args)
        super(MyGraph, self).add_edge(gateway,target)
        for source in sources:
            super(MyGraph, self).add_edge(source, gateway)

    def add_xor_merge_gateway2(self, sources, target, left_style, right_style, *args):
        gateway = 'XORm '+str(sources)+'->'+str(target)
        self.add_xor_gateway(gateway, *args)
        self.add_edge2(gateway,target, right_style)
        for source, style in zip(sources,left_style):
            self.add_edge2(source,gateway,style)
            
    def add_xor_to_and_gateway(self, sources, targets, *args):
        gateway_xor = 'XORm '+str(sources)+'->'+str(targets)
        gateway_and = 'ANDs '+str(sources)+'->'+str(targets)
        self.add_xor_gateway(gateway_xor, *args)
        self.add_and_gateway(gateway_and, *args)
        super(MyGraph, self).add_edge(gateway_xor,gateway_and)
        for source in sources:
            super(MyGraph, self).add_edge(source, gateway_xor)
        for target in targets:
            super(MyGraph, self).add_edge(gateway_and, target)

In [9]:
def get_strict_causality(direct_succession, rhombs):
    causality = defaultdict(set)
    for a, bs in direct_succession.items():
        for b in bs:
            direct_b_a = b in direct_succession and a in direct_succession[b]
            if not direct_b_a or (a,b) in rhombs:
                causality[a].add(b)
            
    return dict(causality)

def get_strict_parallel(direct_succession, events, rhombs): 
    parallel_events = set()
    for a in events:
        for b in events:
            if a == b or a not in direct_succession or b not in direct_succession:
                continue
            if b in direct_succession[a] and a in direct_succession[b] and (a,b) not in rhombs:
                parallel_events.add((a,b))
    return parallel_events

def get_events(logs):
    flat_logs = [item for sublist in logs for item in sublist]
    return set(flat_logs)

def get_events2(direct_succession):
    evs = set()
    for k, vs in direct_succession.items():
        evs.add(k)
        for v in vs:
            evs.add(v)
    return evs

def get_triangles(logs, events):
    triangles = set()
    for log in logs:
        for i in range(len(log)-2):
            a = log[i]
            b = log[i+1]
            c = log[i+2]
            if a == c:
                triangles.add((a,b))
    return triangles
        
def get_rhombs(logs, events):
    triangles = get_triangles(logs,events)
    rhombs = set()
    for (a,b) in triangles:
        if (b,a) in triangles:
            rhombs.add((a,b))
    return rhombs

def get_start_set_events(logs):
    start_set_events = set()
    for trace in logs:
        start_set_events.add(trace[0])
    return start_set_events

def get_end_set_events(logs):
    end_set_events = set()
    for trace in logs:
        end_set_events.add(trace[-1])
    return end_set_events

def add_start_end_activities(logs):
    res = []
    for trace in logs:
        res_trace = ['start']
        for event in trace:
            res_trace.append(event)
        res_trace.append('end')
        res.append(res_trace)
    return res

def all(el1,el2):
    for e in el1:
        if e not in el2:
            return False
    return True

def pairs(pairs_el):
    p = set()
    for a in pairs_el:
        for b in pairs_el:
            if a != b:
                p.add((a,b))
    return p

def first(first_el):
    return list(first_el)[0]


In [10]:
def draw_graph(events):

  # events_tab = []
  # for line in events.splitlines():
  #     events_tab.append(line.split())

  events_tab = events
  start_set_events = get_start_set_events(events_tab)
  end_set_events = get_end_set_events(events_tab)
  direct_succession = get_direct_succession(events_tab)
  parallel_events = get_potential_parallelism(direct_succession)
  causality = get_causality(direct_succession)
  inv_causality = get_inv_causality(causality)

  G = MyGraph()

  # adding start event
  G.add_event("start")
  if len(start_set_events) > 1:
      if tuple(start_set_events) in sorted(parallel_events):
          G.add_and_split_gateway("start",start_set_events)

      else:
          G.add_xor_split_gateway("start",start_set_events)
  else:
      G.add_edge("start",list(start_set_events)[0])

  # adding split gateways based on causality
  for event in causality:
      if len(causality[event]) > 1 and tuple(causality[event]) != tuple(inv_causality.keys()):
          if tuple(causality[event]) in parallel_events:

              to_check = list(causality[event])
              is_event = False
              for eventeee in events_tab:
                if len(to_check) != len(list(set(sorted(to_check)).intersection(sorted(set(eventeee))))):
                  is_event = True

              if is_event:
                G.add_event_gateway(event,causality[event])
              else:
                G.add_and_split_gateway(event,causality[event])
          else:
              G.add_xor_split_gateway(event,causality[event])

  # adding merge gateways based on inverted causality
  for event in inv_causality:
      if len(inv_causality[event]) > 1 and tuple(sorted(inv_causality[event])) != tuple(sorted(causality.keys())):
          if tuple(inv_causality[event]) in parallel_events:
              to_check = list(inv_causality[event])
              is_event = False
              for eventeee in events_tab:
                if len(to_check) != len(list(set(sorted(to_check)).intersection(sorted(set(eventeee))))):
                  is_event = True

              if is_event:
                G.add_event_merge_gateway(inv_causality[event],event)
              else:
                G.add_and_merge_gateway(inv_causality[event],event)

          else:
              G.add_xor_merge_gateway(inv_causality[event],event)
      elif len(inv_causality[event]) == 1:
          source = list(inv_causality[event])[0]
          G.add_edge(source,event)

  xor_merge_gateways = set()
  for event in inv_causality:
    if (len(inv_causality[event]) > 1 and tuple(sorted(inv_causality[event])) == tuple(sorted(causality.keys()))):
      xor_merge_gateways.add(tuple(sorted(inv_causality[event])))

  # print(xor_merge_gateways)
  for create_gateway_event in xor_merge_gateways:
     G.add_xor_to_and_gateway(inv_causality[event],tuple(inv_causality.keys()))

  end_to_delete = False
  for end_event in end_set_events:
      for start_event in start_set_events:
        try:
          if tuple(causality[end_event]) == tuple(start_event):
            G.add_xor_split_gateway(end_event, (start_event,"end"))
            end_to_delete = True
        except:
          print('Except')

  # adding end event
  G.add_end_event("end")
  if len(end_set_events) > 1 and end_to_delete == False:
      if tuple(end_set_events) in parallel_events:
          G.add_and_merge_gateway(end_set_events,"end")
      else:
          G.add_xor_merge_gateway(end_set_events,"end")
  elif end_to_delete == False:
      G.add_edge(list(end_set_events)[0],"end")

  add_edges_list = []
  for sucession in direct_succession:
    found_count = 0
    for test in direct_succession:
      if test != sucession:
        if sucession in direct_succession[test]:
          found_count = found_count + 1
    if(found_count == len(direct_succession) -1):
      add_edges_list.append(sucession);

  for single_causality in causality:
    to_check = causality[single_causality]
    found_count = 1
    for test in causality:
      if test != single_causality:
        if to_check == causality[test]:
          found_count = found_count + 1
    if len(to_check) == 1 and found_count == 1 and list(to_check)[0] in add_edges_list:
      G.add_edge(single_causality,list(to_check)[0])

  G.draw('simple_process_model.png', prog='dot')
  display(Image('simple_process_model.png'))

In [15]:
from more_itertools import pairwise
from collections import Counter
from itertools import chain
import operator

def filter_process_logs(dfs, event_filter = 0, flow_filter = 0):
    ev_counter = dfs.Activity.value_counts()
    dfs = (dfs
        .sort_values(by=['Case ID','Start Timestamp'])
        .groupby(['Case ID'])
        .agg({'Activity': ';'.join})
    )
    dfs['count'] = 0
    dfs = (
        dfs.groupby('Activity', as_index=False).count()
        .sort_values(['count'], ascending=False)
        .reset_index(drop=True)
    )
    dfs['trace'] = [trace.split(';') for trace in dfs['Activity']]
    
    w_net = dict()
    ev_start_set = set()
    ev_end_set = set()
    for index, row in dfs[['trace','count']].iterrows():
        if row['trace'][0] not in ev_start_set:
            ev_start_set.add(row['trace'][0])
        if row['trace'][-1] not in ev_end_set:
            ev_end_set.add(row['trace'][-1])
        for ev_i, ev_j in pairwise(row['trace']):
            if ev_i not in w_net.keys():
                w_net[ev_i] = Counter()
            w_net[ev_i][ev_j] += row['count']
            
    w_net_prev = dict()
    ev_start_set_prev = set()
    ev_end_set_prev = set()
    for index, row in dfs[['trace','count']].iterrows():
        if row['trace'][0] not in ev_start_set_prev:
            ev_start_set_prev.add(row['trace'][0])
        if row['trace'][-1] not in ev_end_set_prev:
            ev_end_set_prev.add(row['trace'][-1])
        for ev_i, ev_j in pairwise(reversed(row['trace'])):
            if ev_i not in w_net_prev.keys():
                w_net_prev[ev_i] = Counter()
            w_net_prev[ev_i][ev_j] += row['count']
            
    trace_counts = sorted(chain(*[c.values() for c in w_net.values()]))
    trace_min = trace_counts[0]
    trace_max = trace_counts[-1]
    color_min = ev_counter.min()
    color_max = ev_counter.max()
    
    nodes = set()
    edges = set()

    for ev_start in ev_start_set:
        edges.add(("start", ev_start, 0, 0))

    for event, succesors in w_net.items():
        value = ev_counter[event]
        if value < event_filter:
          continue

        color = int(float(color_min-value)/float(color_min-color_max)*100.00)
        my_color = "#ff9933"+str(hex(color))[2:]

        label = str(event) + ": " + str(ev_counter[event])
        nodes.add((event, my_color, label))

        for succesor, cnt in succesors.items():
            if cnt < flow_filter:
              continue
            edges.add((event, succesor, 4*cnt/(trace_max-trace_min)+0.1, cnt))

            succesor, cnt = max(succesors.items(), key=operator.itemgetter(1))
            edges.add((event, succesor, 4*cnt/(trace_max-trace_min)+0.1, cnt))
    
    for event, predecessors in w_net_prev.items():
        predecessors, cnt = max(predecessors.items(), key=operator.itemgetter(1))
        edges.add((predecessors, event, 4*cnt/(trace_max-trace_min)+0.1, cnt))


    for ev_end in ev_end_set:
        edges.add((ev_end, "end", 0, 0))
    
    direct_succession = defaultdict(set)
    
    for (a,b,width,count) in edges:
        direct_succession[a].add(b)       

    return nodes, edges, direct_succession, trace_min, trace_max, color_min, color_max 

In [13]:
def filter_edges(A, B, edges):
    res = set()
    for (a,b,c,d) in edges:
        if a in A and b in B:
            res.add((a,b,c,d))
    return res

def edge_sum(edges, trace_max, trace_min):
    cnt = 0
    for (c,d) in edges:
        cnt += d
    width = 4*cnt/(trace_max - trace_min)+0.1
    return (width, cnt)

def tuple_array_slice(arr, sli):
    res = []
    for e in arr:
        res.append(e[sli])
    return res
    

def draw_filtered_model(logs, traces, event_filter = 0, flow_filter = 0):  
    nodes, edges, direct_succession, trace_min, trace_max, color_min, color_max  = filter_process_logs(logs, event_filter, flow_filter)

    traces = add_start_end_activities(traces)

    events = get_events2(direct_succession)
    
    rhombs = get_rhombs(traces,events)
    causality = get_strict_causality(direct_succession, rhombs)
    inv_causality = get_inv_causality(causality)
    
    parallel_events = get_potential_parallelism(direct_succession)

    
    G = MyGraph()
    
    new = []

    G.add_event("start")
    G.add_end_event("end")
    
    for (name, color, label) in nodes:
        G.add_event2(name, style="rounded,filled", fillcolor=color, label=label)
    
    for event in causality:
        if len(causality[event]) > 1:
            if all(pairs(causality[event]), parallel_events):        
                new.append(({event}, causality[event], 'and'))
            else:
                new.append(({event}, causality[event], 'xor'))

    for event in inv_causality:
        if len(inv_causality[event]) > 1:
            if all(pairs(inv_causality[event]), parallel_events):
                new.append((inv_causality[event], {event}, 'and'))
            else:
                new.append((inv_causality[event], {event}, 'xor'))
        elif len(inv_causality[event]) == 1:
            source = list(inv_causality[event])[0]
            new.append(({source}, {event}, 'edge'))

         
    for a, b in causality.items():
        if len(b) == 1:
            already_contained = False
            for (a1,b1,typ) in new:
                if a in a1:
                    already_contained = True
                    break
            if not already_contained:
                new.append(({a},b, 'edge')) 
            
    for (a,b,typ) in new:
        rel_edges = filter_edges(a, b, edges)
        rel_edges = tuple_array_slice(rel_edges, slice(2, None))
        sum_edge = edge_sum(rel_edges, trace_max, trace_min)

        if len(b) > 1:
            if typ == 'and':
                G.add_and_split_gateway2(first(a),b,sum_edge,rel_edges)                
            else:
                G.add_xor_split_gateway2(first(a),b,sum_edge,rel_edges)
        elif len(a) > 1:
            if typ == 'and':
                G.add_and_merge_gateway2(a,first(b),rel_edges,sum_edge)
            else:
                G.add_xor_merge_gateway2(a,first(b),rel_edges,sum_edge)
        else:
            G.add_edge(first(a),first(b))

    G.draw(name + ".png", prog='dot')
    display(Image(name + ".png"))
    
logs = load_csv(file_name)



Enter the name of timestamp column: Start Timestamp
Enter the name of activity column: Activity
Start Timestamp
Case ID
Activity


In [16]:
import ipywidgets as widgets
from ipywidgets import interact

def draw_interact_graph(event_filter, flow_filter):
    draw_filtered_model(logs[1], logs[0], event_filter, flow_filter)

interact(draw_interact_graph, event_filter = event_filter,flow_filter = flow_filter)

interactive(children=(IntSlider(value=700, description='event_filter', max=1200), IntSlider(value=700, descrip…

<function __main__.draw_interact_graph>